In [1]:
import librosa
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import audio2numpy as an


test_dataset = load_dataset("common_voice", "fi", split="test[:2%]")

processor = Wav2Vec2Processor.from_pretrained("aapot/wav2vec2-large-xlsr-53-finnish")
model = Wav2Vec2ForCTC.from_pretrained("aapot/wav2vec2-large-xlsr-53-finnish")

resampler = lambda sr, y: librosa.resample(y.numpy().squeeze(), sr, 16_000)

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = an.open_audio(batch["path"])
    speech_array = torch.tensor(speech_array)
    batch["speech"] = resampler(sampling_rate, speech_array).squeeze()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits


    
predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])

C:\Users\immon\anaconda3\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
Reusing dataset common_voice (C:\Users\immon\.cache\huggingface\datasets\common_voice\fi\6.1.0\0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Loading cached processed dataset at C:\Users\immon\.cache\huggingface\datasets\common_voice\fi\6.1.0\0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f\cache-e34306c9680dc3af.arrow


Prediction: ['mysteerimies oli oppinut morallinsa taruista elokuvista ja peleistä', 'äänestin mietinnön puolesta n']
Reference: ['Mysteerimies oli oppinut moraalinsa taruista, elokuvista ja peleistä.', 'Äänestin mietinnön puolesta!']


In [2]:
class SpeechRecognizer:
    
    def __init__(self, model, processor, device="cuda"):
        self.model = model
        model.to(device)
        self.processor = processor
        self.device = device
        
    def _prepareaudio(self, path : str):
        # load audio
        # regex
        audio, sr = an.open_audio(path)
        try:
            audio = audio[:, 1]
        except:
            pass
        audio = torch.tensor(audio)
        audio = librosa.resample(audio.numpy().squeeze(), sr, 16_000)
        return audio
    
    def decode(self, output: torch.tensor, mode: str = "argmax"):
        
        if mode=="argmax":
            pred_ids = torch.argmax(output, dim=-1)
            
        return pred_ids
    
    
    @torch.no_grad()
    def __call__(self, path: str):
        audio = self._prepareaudio(path)
        inputs = processor(audio, sampling_rate=16_000, return_tensors="pt", padding=True)
        logits = model(inputs.input_values.to(self.device), attention_mask=inputs.attention_mask.to(self.device)).logits
        pred_ids = self.decode(logits)
        prediction = processor.batch_decode(pred_ids)
        return prediction, logits
    
recog = SpeechRecognizer(model, processor, device="cuda")

pred, logits = recog("data/1aaefe12-a79e-493f-9dff-c238482632f2.mp3")
logits = logits.squeeze(0)
#pred = torch.argmax(logits[:,:, :], dim=-1)
#pred = processor.batch_decode(pred)
print(pred)

['onko olemassa jonkin laista dokumenttiheiänohjalmasta']


In [3]:
#help(processor.tokenizer)
vocab = processor.tokenizer.get_vocab()
print(vocab)

{'b': 0, 'y': 1, 'w': 2, 'u': 3, 'a': 4, 'r': 5, 'k': 6, 'e': 7, 'ö': 8, 'j': 9, 'm': 11, 'ä': 12, 'i': 13, 'h': 14, 'x': 15, 't': 16, 'z': 17, 'l': 18, 'c': 19, 'v': 20, 'f': 21, 'p': 22, 'q': 23, 'o': 24, 'n': 25, 'd': 26, 's': 27, 'g': 28, '|': 10, '[UNK]': 29, '[PAD]': 30, '<s>': 31, '</s>': 32}


# argmax decoder
1. first take argmax
2. merge repeats 
3. remove blanks
4. map ids to letters
5. change word separation to space

In [4]:
from itertools import groupby
import numpy as np
def invert_dict(dict):
    return {v: k for k, v in dict.items()}

dd = invert_dict(vocab)

def argmax_decode(probs, vocab, blank_id = 30):
    """
    argmax decoder for ctc
    uses a specific vocab
    
    inputs: probs: torch tensor of shape T,V
            vocab: vocab mapping ids to characters
            blank_id: the id of a blank token
            
    output: decoded sequence of characters
    
    """
    #argmax
    pred_ids = torch.argmax(probs, dim=-1)
    pred_ids = pred_ids.numpy()
    #merge repeats
    merged = np.array([k for k,_ in groupby(pred_ids)])
    #remove blanks
    result = merged[merged!=30]
    #map with dictionary
    result = [dd[i] for i in result]
    #remove extra spaces
    result = result
    
    return "".join(result).replace('|', ' ')
    
    
argmax_decode(logits, dd)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [5]:
vals = sorted(vocab.items(), key = lambda x:x[1])
vals

[('b', 0),
 ('y', 1),
 ('w', 2),
 ('u', 3),
 ('a', 4),
 ('r', 5),
 ('k', 6),
 ('e', 7),
 ('ö', 8),
 ('j', 9),
 ('|', 10),
 ('m', 11),
 ('ä', 12),
 ('i', 13),
 ('h', 14),
 ('x', 15),
 ('t', 16),
 ('z', 17),
 ('l', 18),
 ('c', 19),
 ('v', 20),
 ('f', 21),
 ('p', 22),
 ('q', 23),
 ('o', 24),
 ('n', 25),
 ('d', 26),
 ('s', 27),
 ('g', 28),
 ('[UNK]', 29),
 ('[PAD]', 30),
 ('<s>', 31),
 ('</s>', 32)]

In [6]:
import collections
NEG_INF = -float("inf")


def map_to_chars(ids, vocab):
    result = [vocab[i] for i in ids]
    return "".join(result).replace('|', ' ')

In [7]:
import IPython.display as ipd
ipd.Audio("data/53a20c7a-dc9d-4bcc-a9fd-531d4e9858e2.mp3")

In [25]:
from sample_decode import decode
import time

t1 = time.time()
pred, logits = recog("data/31f7bb06-b9f2-41b8-9d0d-edb956fa1a97.mp3")
t2 = time.time()
logits = logits.squeeze(0)
print(logits.cpu().numpy().shape)
t3 = time.time()
ids = decode(logits.softmax(dim=1).cpu().numpy(), beam_size=10, blank=30, prune_t = 0.00001)
t4 = time.time()
print(map_to_chars(ids[0], dd))
print(argmax_decode(logits.cpu(), dd))
print(t2-t1)
print(t4-t3)

(234, 31)
vaan julja kakskit kaksivuvuotias sekoikeana piskeli iväiskyldän yliopistosta 
vaan julja kakskit kaksitvuvuotias sekoikeana piskeli iväskyldän yliopistosta 
0.20743775367736816
0.05203056335449219
